In [5]:
import json
from  pathlib import Path

In [6]:
path = Path("../resources/ufrpe/qa.txt")
with open(path, 'r') as f:
    qa = []
    for line in f.readlines():
        qa.append(json.loads(line))

In [7]:
with open('../resources/documento_oficial.json') as json_file:
    documento_oficial = json.load(json_file)

In [28]:
for quesito in qa:
    print(quesito["pergunta"])

Quando começam as aulas do PLE?
Nem o SIGAA nem o AVA podem transmitir as aulas online. Temos na UFRPE uma plataforma para isso?
O/a docente pode decidir se as atividades serão síncronas ou assíncronas?
As aulas serão disponibilizadas no AVA, SIGAA ou em alguma outra plataforma?
Como funcionarão as aulas práticas?
O professor pode propor atividades em que os estudantes tenham que sair de casa (ex.: realização de filmagens ou fotografias em área externa)? 
O/a estudante pode levar falta ou ter a nota alterada por não comparecer a uma atividade síncrona, mesmo que seja uma arguição oral?
Qual é o horário em que ocorrerão as aulas remotas?
Quem pode ofertar disciplinas no PLE?
Como vai funcionar o PLE?
Quem pode cursar disciplinas no PLE?
Por que não há um período específico para a realização das atividades avaliativas no PLE?
Como fica a situação dos estudantes de 2020.1 e 2020.2 com essas propostas?
Por que foi colocado um limite de vagas nas disciplinas?
O cancelamento de matrícula nas

In [19]:
for k,v in documento_oficial.items():
    for kk, vv in v.items():
        for vvv in vv:
            print(vvv)
            print("-"*50)


 este regulamento tem por finalidade consolidar a normatizaçãodo período letivo excepcional (ple) no âmbito da graduação na universidade federal rural depernambuco (ufrpe), em função da suspensão das atividades presenciais devido à pandemia docovid-19. 1º para os efeitos deste regulamento, considera-se ple aquelecaracterizado pelas atividades acadêmicas, no âmbito da graduação, realizadas por meio deensino remoto durante suspensão de atividades presenciais em virtude da pandemia do novocoronavírus (covid-19). 2º compreende-se ensino remoto como a realização de atividades deensino mediadas pelo uso de tecnologias, caracterizadas pelo conjunto de ações de ensino-aprendizagem ou atos de currículo mediados por interfaces digitais que potencializam práticascomunicacionais interativas e hipertextuais, formato distinto da educação a distância (ead),compreendida como modalidade educacional que prioriza a mediação didático-pedagógica pormeio de tecnologias digitais de informação e comunicação (